In [18]:
import at
from pySC.core.simulated_commissioning import SimulatedCommissioning
from pySC.utils.sc_tools import SCgetOrds
from pySC.plotting.plot_lattice import plot_lattice
from pySC.utils import logging_tools
from pySC.correction.loco_modules import *
from matplotlib import pyplot as plt
from pySC.lattice_properties.response_measurement import response_matrix
import copy
from at import Lattice
LOGGER = logging_tools.get_logger(__name__)
def create_at_lattice() -> Lattice:
    def _marker(name):
        return at.Marker(name, PassMethod='IdentityPass')
    new_ring = at.load_mat('hmba.mat')
    bpm_indexes = at.get_refpts(new_ring,at.elements.Monitor)
    for i in reversed(bpm_indexes):
        Cor = at.elements.Corrector('CXY'+str(i),  length = 0, kick_angle= [0, 0], PolynomA=[0,0] , PolynomB=[0,0])
        new_ring.insert(i+1, Cor)
    new_ring.enable_6d()
    at.set_cavity_phase(new_ring)
    at.set_rf_frequency(new_ring)
    new_ring.tapering(niter=3, quadrupole=True, sextupole=True)

    return new_ring

if __name__ == "__main__":
    ring = at.Lattice(create_at_lattice())
    LOGGER.info(f"{len(ring)=}")
    SC = SimulatedCommissioning(ring)
    ords = SCgetOrds(SC.RING, 'BPM')
    SC.register_bpms(ords,
                     Roll=0.0, CalError=1E-2 * np.ones(2))
    ords = SCgetOrds(SC.RING, 'QF')
    SC.register_magnets(ords, # [rad]
                        CalErrorB=np.array([0, 1E-2]))  # relative
    ords = SCgetOrds(SC.RING, 'QD')
    SC.register_magnets(ords, # [rad]
                        CalErrorB=np.array([0, 1E-2]))#

    ords = SCgetOrds(SC.RING, 'CXY')
    SC.register_magnets(ords, CalErrorA=np.array([1E-200, 0]),
                        CalErrorB=np.array([1E-200, 0]))

    ords = SCgetOrds(SC.RING, 'BEND')
    SC.register_magnets(ords)
    ords = SCgetOrds(SC.RING, 'SF|SD')
    SC.register_magnets(ords)  # [1/m]
    ords = SCgetOrds(SC.RING, 'RFC')
    SC.register_cavities(ords)
    SC.apply_errors()

CorOrds  = SCgetOrds(SC.RING, 'CXY')

used_correctors1 = select_equally_spaced_elements(CorOrds, 10)
used_correctors2 = select_equally_spaced_elements(CorOrds, 10)
CorOrds = [used_correctors1,used_correctors2]
used_bpm1 = select_equally_spaced_elements(SC.ORD.BPM, 10)
used_bpm2 = select_equally_spaced_elements(SC.ORD.BPM, 10)
used_bpms = [used_bpm1,used_bpm2]

CAVords = SCgetOrds(SC.RING, 'RFC')
quadsOrds = [SCgetOrds(SC.RING, 'QF'), SCgetOrds(SC.RING, 'QD')]
CAVords = SCgetOrds(SC.RING, 'RFCav')
sextOrds = SCgetOrds(SC.RING, 'SF|SD')
skewOrds =  SC.ORD.SkewQuad
CMstep =  np.array([1.e-4]) #correctors change [rad]
dk = 1.e-4 #quads change
RFstep = 1e3

_, _, twiss = at.get_optics(SC.IDEALRING, SC.ORD.BPM)
orbit_response_matrix_model = SCgetModelRM(SC, SC.ORD.BPM, CorOrds, trackMode='ORB', useIdealRing=True, dkick= CMstep)
ModelDispersion = SCgetModelDispersion(SC, SC.ORD.BPM, CAVords, trackMode='ORB', Z0=np.zeros(6), nTurns=1, rfStep=RFstep, useIdealRing=True)
errq =[]
for i in np.concatenate(quadsOrds):
    errq.append(SC.RING[i].K- SC.IDEALRING[i].K)

err_cor=[]
for i in CorOrds[0]:
    err_cor.append(SC.RING[i].CalErrorB[0])
for i in CorOrds[1]:
    err_cor.append(SC.RING[i].CalErrorA[0])
orbit_response_matrix_measured = SCgetMeasurRM(SC, SC.ORD.BPM, CorOrds, CMstep)
_, _, twiss_err = at.get_optics(SC.RING, SC.ORD.BPM)

Jn = generatingJacobian(SC, orbit_response_matrix_model, CMstep, CorOrds, SC.ORD.BPM, np.concatenate(quadsOrds), dk, debug=True, trackMode='ORB', useIdealRing=False,skewness = False, order=1, method='add', includeDispersion=False, rf_step=RFstep, cav_ords=CAVords )

err_cor=[]
for i in CorOrds[0]:
    err_cor.append(SC.RING[i].CalErrorB[0])
for i in CorOrds[1]:
    err_cor.append(SC.RING[i].CalErrorA[0])
orbit_response_matrix_measured = SCgetMeasurRM(SC, SC.ORD.BPM, CorOrds, CMstep)


err_bpms=[]
for i in SC.ORD.BPM:
    err_bpms.append(SC.RING[i].CalError[0])
for i in SC.ORD.BPM:
    err_bpms.append(SC.RING[i].CalError[1])

   INFO | len(ring)=131 | ipykernel_14552.2914558345
   INFO | Circumference error applied. | core.simulated_commissioning
   INFO | Calculating model response matrix | lattice_properties.response_model
   INFO | Calculating model dispersion | lattice_properties.response_model
Calculating Measure response matrix
Calculating Measure response matrix


In [19]:
numberOfIteration = 1
sCut = 16
W = 1
for x in range(numberOfIteration): # optics correction using QF and QD
    print('LOCO iteration ', x)

    C_measure = SCgetMeasurRM(SC, SC.ORD.BPM, CorOrds, CMstep)
    bx_rms_err, by_rms_err = getBetaBeat(SC.RING, twiss, SC.ORD.BPM, makeplot=False)
    Jn = np.transpose(Jn, (0, 2, 1))
    Jt = getInverse(Jn, sCut, W)

    quads =  len(np.concatenate(quadsOrds))
    cor =  len(np.concatenate(CorOrds))
    bpm =  len(SC.ORD.BPM) *2

    total_length = bpm + cor +quads
    lengths = [quads ,cor,  bpm]
    including_fit_parameters = ['quads', 'cor', 'bpm']
    initial_guess = np.zeros(total_length)
    initial_guess[:lengths[0]]= 1e-6
    initial_guess[lengths[0]:lengths[0] + lengths[1] ]= 1e-6
    initial_guess[lengths[0] + lengths[1] :] = 1e-6


    fit_parameters  = loco_correction(lambda delta_params: objective(delta_params, np.transpose(orbit_response_matrix_model), np.transpose(orbit_response_matrix_measured), Jn, lengths, including_fit_parameters, W),initial_guess,np.transpose(orbit_response_matrix_model), np.transpose(orbit_response_matrix_measured), Jn,Jt, lengths,including_fit_parameters
                         , verbose=2, max_iterations= 100, eps=1e-6, method='lm',W= W)

    dg  = fit_parameters[:lengths[0]]
    dx = fit_parameters[lengths[0]:lengths[0] + lengths[1] ]
    dy = fit_parameters[lengths[0] + lengths[1] :]
    print('SVD')
    SC = setCorrection_(SC,dg, np.concatenate(quadsOrds))
    _, _, twiss_corr = at.get_optics(SC.RING, SC.ORD.BPM)
    bx_rms_cor, by_rms_cor = getBetaBeat(SC.RING, twiss, SC.ORD.BPM, makeplot=False)
    print(
        "Before LOCO correction:\n"
        f"RMS horizontal beta beating: {bx_rms_err*100:.2f}%   RMS vertical beta beating: {by_rms_err*100:.2f}%\n"

        f"After LOCO corrections\n"
        f"RMS horizontal beta beating: {bx_rms_cor*100:.2f}%   RMS vertical beta beating: {by_rms_cor*100:.2f}%\n"
        f"beta_x correction reduction: {(1 - bx_rms_cor / bx_rms_err) * 100:.2f}%\n"
        f"beta_y correction reduction: {(1 - by_rms_cor / by_rms_err) * 100:.2f}%\n "

    )

LOCO iteration  0
Calculating Measure response matrix
`ftol` termination condition is satisfied.
Function evaluations 582, initial cost 1.8871e+00, final cost 5.4577e-02, first-order optimality 5.16e-06.
SVD
Before LOCO correction:
RMS horizontal beta beating: 6.59%   RMS vertical beta beating: 5.13%
After LOCO corrections
RMS horizontal beta beating: 1.08%   RMS vertical beta beating: 0.61%
beta_x correction reduction: 83.65%
beta_y correction reduction: 88.16%
 
